In [13]:
import hdbscan
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import load_npz
clusterer = hdbscan.HDBSCAN(min_cluster_size=64, metric='euclidean')

In [14]:
X = load_npz("../data_preprocessing/X_tfidf_sparse_1_2_min_df_5.npz")

In [15]:
import hdbscan
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=30)
X_reduced = svd.fit_transform(X.T) # 注意轉置 → 每個 row 是一部電影

In [16]:
# clusterer = hdbscan.HDBSCAN(min_cluster_size=20)
labels = clusterer.fit_predict(X_reduced)  # shape: (num_movies,)

/Users/yuu/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/yuu/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [17]:
import numpy as np
np.save("cluster_labels_1_2_min_df_5.npy", labels)